<a href="https://colab.research.google.com/github/zhaomargot/energyloadforecasts/blob/main/zhaomargot_dso424_goal2_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Margot Zhao

In [ ]:
# install pycaret
!pip install pycaret
# import libraries
import pandas as pd
from pycaret.time_series import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf
import holidays


# resolution
mpl.rcParams['figure.dpi'] = 300
plt.style.use('seaborn-whitegrid')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import data as pandas file
ogdf = pd.read_excel("https://github.com/robertasgabrys/DSO424SPRING2023/blob/main/CompetitionData.xlsx?raw=true")

In [ ]:
# create timestamps
ogdf["Time"] = pd.date_range(start = pd.Timestamp('2002-01-01 00:00'),end = pd.Timestamp('2006-12-31 23:59'),freq='H')

# Create a column "Day" with just the date and not time
ogdf["Day"] = ogdf["Time"].dt.date

print(ogdf.head())

   Tavg  Tmed  Tmax  Tmin       Load                Time         Day
0    43  43.0  60.0  31.0  1384494.0 2002-01-01 00:00:00  2002-01-01
1    42  42.0  58.0  29.0  1392822.0 2002-01-01 01:00:00  2002-01-01
2    41  41.0  57.0  31.0  1407887.0 2002-01-01 02:00:00  2002-01-01
3    41  41.0  56.0  30.0  1438658.0 2002-01-01 03:00:00  2002-01-01
4    40  41.0  53.0  29.0  1484046.0 2002-01-01 04:00:00  2002-01-01


In [ ]:
# Get average values of temperature columns, max value of load per day
df = ogdf.groupby("Day").agg({"Tavg": "mean", "Tmed": "mean", "Tmax": "mean", "Tmin": "mean", "Load": "max"})[["Tavg", "Tmed", "Tmax", "Tmin", "Load"]].reset_index()
print(df)

             Day       Tavg       Tmed       Tmax       Tmin       Load
0     2002-01-01  48.500000  49.125000  61.125000  36.166667  1871982.0
1     2002-01-02  51.291667  54.916667  64.041667  36.791667  1627644.0
2     2002-01-03  43.625000  42.791667  68.708333  34.541667  2260145.0
3     2002-01-04  37.375000  37.666667  46.416667  30.333333  2545548.0
4     2002-01-05  45.500000  46.083333  64.458333  31.750000  2588347.0
...          ...        ...        ...        ...        ...        ...
1821  2006-12-27  47.291667  47.375000  58.750000  40.041667        NaN
1822  2006-12-28  57.041667  58.291667  70.375000  46.791667        NaN
1823  2006-12-29  65.541667  65.583333  75.125000  58.541667        NaN
1824  2006-12-30  69.166667  69.708333  76.500000  61.875000        NaN
1825  2006-12-31  71.541667  72.208333  77.083333  64.333333        NaN

[1826 rows x 6 columns]


In [ ]:
#Find the hour at which the daily peak occurs and append to dataframe
peakseries = ogdf.groupby(["Day"])["Load"].max()
# Turn into a dataframe
peakdf = pd.DataFrame({"Day": peakseries.index, "Load": peakseries.values})
# Column "Time" from df3 now split into two columns, "Day" and "Hour"
ogdf["Hour"] = ogdf["Time"].dt.time

# Use an inner join to preserve Tavg / Tmed / Tmax / Tmin / Hour data for the daily peak
peakdf = peakdf.merge(ogdf, how = 'inner', on = ["Day", "Load"])
# Get hour as a number
peakdf["Hour"] = peakdf["Time"].dt.strftime("%H")
peakdf["Hour"] = peakdf["Hour"].astype(int)

# append hour to DF
df["Hour"] = peakdf["Hour"]
df["Time"] = peakdf["Time"]

print("DF: \n", df)

DF: 
              Day       Tavg       Tmed       Tmax       Tmin       Load  Hour  \
0     2002-01-01  48.500000  49.125000  61.125000  36.166667  1871982.0     8   
1     2002-01-02  51.291667  54.916667  64.041667  36.791667  1627644.0    18   
2     2002-01-03  43.625000  42.791667  68.708333  34.541667  2260145.0    20   
3     2002-01-04  37.375000  37.666667  46.416667  30.333333  2545548.0     7   
4     2002-01-05  45.500000  46.083333  64.458333  31.750000  2588347.0     8   
...          ...        ...        ...        ...        ...        ...   ...   
1821  2006-12-27  47.291667  47.375000  58.750000  40.041667        NaN     0   
1822  2006-12-28  57.041667  58.291667  70.375000  46.791667        NaN     1   
1823  2006-12-29  65.541667  65.583333  75.125000  58.541667        NaN     2   
1824  2006-12-30  69.166667  69.708333  76.500000  61.875000        NaN     3   
1825  2006-12-31  71.541667  72.208333  77.083333  64.333333        NaN     4   

                    T

In [ ]:
# Seasonality
df["Month"] = [i.month for i in df["Day"]]
df["Year"] = [i.year for i in df["Day"]]
df["Day_of_week"] = [i.isoweekday() for i in df["Day"]]
df["Day_of_year"] = [pd.Period(i, freq='D').day_of_year for i in df["Day"]]

df.set_index("Day", inplace=True)
df.index =  pd.to_datetime(df.index, format='%Y-%m-%d')

# for training models, create df3 with only data from 2002 to 2005
df3 = df.loc[df["Time"].dt.year < 2006]

# drop time & hour
df3.drop(columns=["Time", "Hour"], inplace=True)


In [ ]:
df3.reset_index(drop=True, inplace=True)


In [ ]:
 # train size (2002-2004) = 1096 days out of 1461 = 0.750171116,
 from pycaret.regression import *
 s = setup(df3, target = "Load", train_size=0.750171116,
              data_split_shuffle = False, fold_strategy = 'timeseries', fold = 3,
              numeric_features = ['Day_of_year', "Tmax", "Tmin", "Year", "Tavg", "Tmed",],
              categorical_features = ['Month', 'Day_of_week'], n_features_to_select=2, transformation=True, verbose = False, use_gpu=True, session_id = 123)


In [ ]:
best_model = compare_models(sort = 'MAPE', verbose=False)

In [ ]:
all_results = []
p = pull().iloc[0:1]
all_results.append(p)
print(p)
# MAPE = 0.0737

                                    Model         MAE           MSE  \
lightgbm  Light Gradient Boosting Machine  134489.137  3.342916e+10   

                 RMSE      R2  RMSLE    MAPE  TT (Sec)  
lightgbm  180354.4144  0.7446  0.098  0.0737    0.1767  


In [ ]:
best = create_model("lightgbm", fold=3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,153750.5579,46469175666.3286,215567.1025,0.6593,0.1159,0.0865
1,106714.7917,20220832469.4345,142199.9735,0.8231,0.0794,0.0597
2,143002.0614,33597484881.8077,183296.1671,0.7512,0.0989,0.0749
Mean,134489.1370,33429164339.1902,180354.4144,0.7446,0.0980,0.0737
Std,20123.6716,10716502196.7457,30024.1496,0.0670,0.0149,0.0110


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
tuned_best = tune_model(best, optimize='MAPE', n_iter=5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,213446.7655,96436192811.3727,310541.7731,0.2931,0.1718,0.1185
1,109527.4167,21128297739.2232,145355.7627,0.8152,0.0800,0.0614
2,137591.8773,32070697826.6530,179082.9356,0.7625,0.0960,0.0719
Mean,153522.0198,49878396125.7496,211660.1571,0.6236,0.1159,0.0839
Std,43894.8353,33223037730.6993,71262.7113,0.2347,0.0401,0.0248


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 5 candidates, totalling 15 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
#2005 Forecast
prediction_holdout = predict_model(best)
#MAPE: 0.0710

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,142640.2734,33306433070.8590,182500.5016,0.8134,0.0918,0.0710


In [ ]:
#2006 Forecast
final = finalize_model(best)

In [ ]:
print(df)
df.drop(columns=["Time", "Load"], inplace=True)
df.reset_index(drop=True, inplace=True)
df2006 = df.drop(df3.index)
df2006.reset_index(drop=True, inplace=True)
print(df2006)

                 Tavg       Tmed       Tmax       Tmin       Load  Hour  \
Day                                                                       
2002-01-01  48.500000  49.125000  61.125000  36.166667  1871982.0     8   
2002-01-02  51.291667  54.916667  64.041667  36.791667  1627644.0    18   
2002-01-03  43.625000  42.791667  68.708333  34.541667  2260145.0    20   
2002-01-04  37.375000  37.666667  46.416667  30.333333  2545548.0     7   
2002-01-05  45.500000  46.083333  64.458333  31.750000  2588347.0     8   
...               ...        ...        ...        ...        ...   ...   
2006-12-27  47.291667  47.375000  58.750000  40.041667        NaN     0   
2006-12-28  57.041667  58.291667  70.375000  46.791667        NaN     1   
2006-12-29  65.541667  65.583333  75.125000  58.541667        NaN     2   
2006-12-30  69.166667  69.708333  76.500000  61.875000        NaN     3   
2006-12-31  71.541667  72.208333  77.083333  64.333333        NaN     4   

                        

In [ ]:
predict_model(final, df2006, verbose=False)

,Tavg,Tmed,Tmax,Tmin,Hour,Month,Year,Day_of_week,Day_of_year,prediction_label
0,66.208336,67.125000,73.791664,58.666668,0,1,2006,7,1,1.411051e+06
1,70.333336,71.458336,75.416664,63.208332,1,1,2006,1,2,1.555774e+06
2,66.625000,67.833336,74.458336,59.000000,2,1,2006,2,3,1.401765e+06
3,57.250000,57.333332,66.791664,49.291668,3,1,2006,3,4,1.664051e+06
4,59.041668,60.083332,65.583336,52.041668,4,1,2006,4,5,1.592812e+06
...,...,...,...,...,...,...,...,...,...,...
360,47.291668,47.375000,58.750000,40.041668,0,12,2006,3,361,2.416007e+06
361,57.041668,58.291668,70.375000,46.791668,1,12,2006,4,362,1.777064e+06
362,65.541664,65.583336,75.125000,58.541668,2,12,2006,5,363,1.511426e+06
363,69.166664,69.708336,76.500000,61.875000,3,12,2006,6,364,1.585709e+06
